In [1]:
import os
import re
import numpy as np
import csv

In [2]:

# 1. Функция токенизации: перевод в нижний регистр и выделение слов (учитывая буквы кириллицы)
def tokenize(text):
    text = text.lower()
    # регулярное выражение ищет последовательности букв и цифр (\w в Unicode)
    tokens = re.findall(r'\b\w+\b', text, flags=re.UNICODE)
    return tokens

In [ ]:
# 2. Загрузка данных из папки dataset/ (подкаталоги: neg, neu, pos)
data = []  # список словарей с ключами: 'filename', 'label', 'text'
dataset_path = "C:/Users/szlat/Hates-speech-detection/dataset"  # имя папки с данными
# Если требуется, можно задать соответствие меток, например:
label_map = {
    "neg": "negative",
    "neu": "neutral",
    "pos": "positive"
}

for folder in ["neg", "neu", "pos"]:
    folder_path = os.path.join(dataset_path, folder)
    # Обработка каждого файла с расширением .txt
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                text = f.read()
            data.append({
                "filename": filename,
                "label": label_map.get(folder, folder),
                "text": text
            })

In [ ]:
# 3. Токенизация текстов и построение словаря
word_counts = {}  # подсчёт частот для каждого слова
for item in data:
    tokens = tokenize(item["text"])
    item["tokens"] = tokens  # сохраняем список токенов для последующего использования
    for token in tokens:
        word_counts[token] = word_counts.get(token, 0) + 1

# Можно при желании отсеять редкие слова (например, min_count=1 оставит все)
min_count = 1
vocab = {word: idx for idx, (word, count) in enumerate(word_counts.items()) if count >= min_count}
vocab_size = len(vocab)
print(f"Размер словаря: {vocab_size}")

In [ ]:

# 4. Формирование обучающих примеров для модели CBOW
# Для каждого слова в документе используем context window (например, по 2 слова слева и справа)
training_samples = []  # список кортежей: ([индексы контекста], target_index)
window_size = 2

for item in data:
    tokens = item["tokens"]
    # для каждого слова в предложении
    for i in range(len(tokens)):
        target_word = tokens[i]
        # если слово не в словаре (хотя по построению все должны быть в vocab), пропускаем
        if target_word not in vocab:
            continue
        target_idx = vocab[target_word]
        context_indices = []
        # определяем границы окна с учётом начала/конца документа
        for j in range(max(0, i - window_size), min(len(tokens), i + window_size + 1)):
            if j == i:
                continue  # пропускаем целевое слово
            word = tokens[j]
            if word in vocab:
                context_indices.append(vocab[word])
        if len(context_indices) > 0:
            training_samples.append((context_indices, target_idx))

print(f"Количество обучающих примеров: {len(training_samples)}")


KeyboardInterrupt: 

In [ ]:
# 5. Инициализация параметров для модели CBOW
embedding_dim = 50  # размерность эмбеддингов (гиперпараметр)
# Матрица эмбеддингов для входных слов: размер (vocab_size, embedding_dim)
W1 = np.random.rand(vocab_size, embedding_dim) - 0.5
# Матрица для выходного слоя: размер (embedding_dim, vocab_size)
W2 = np.random.rand(embedding_dim, vocab_size) - 0.5

In [ ]:

# Функция softmax
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / np.sum(e_x)

In [ ]:
# 6. Обучение модели CBOW (полный softmax; для небольшого словаря и датасета – вполне приемлемо)
lr = 0.01     # скорость обучения
epochs = 5    # число эпох

for epoch in range(epochs):
    total_loss = 0
    # перемешиваем обучающие примеры
    np.random.shuffle(training_samples)
    
    for context_indices, target_idx in training_samples:
        # Вычисляем усредненное представление контекста:
        v_context = np.mean(W1[context_indices], axis=0)  # shape: (embedding_dim,)
        
        # Вычисляем "сырые" оценки (logits) для каждого слова словаря
        z = np.dot(v_context, W2)  # shape: (vocab_size,)
        y_pred = softmax(z)         # прогноз распределения вероятностей
        
        # Вычисление функции потерь (кросс-энтропия)
        loss = -np.log(y_pred[target_idx] + 1e-9)
        total_loss += loss
        
        # Вычисляем градиент ошибки:
        # Начинаем с разности предсказанного распределения и истинного one-hot вектора
        dz = y_pred.copy()
        dz[target_idx] -= 1  # градиент по выходу
        
        # Градиенты для W2: outer product от вектора контекста и dz
        dW2 = np.outer(v_context, dz)  # shape: (embedding_dim, vocab_size)
        
        # Градиент для усредненного вектора контекста:
        dv = np.dot(W2, dz)  # shape: (embedding_dim,)
        
        # Так как v_context = average(W1[context_indices]),
        # градиент для каждого слова из контекста равен dv / (число слов в контексте)
        dv_each = dv / len(context_indices)
        
        # Обновляем веса для слов из контекста
        for idx in context_indices:
            W1[idx] -= lr * dv_each
        
        # Обновляем веса выходного слоя
        W2 -= lr * dW2

    avg_loss = total_loss / len(training_samples)
    print(f"Эпоха {epoch+1}/{epochs}, средняя потеря: {avg_loss:.4f}")



In [ ]:
# 7. После обучения матрица W1 содержит эмбеддинги для каждого слова.
# Можно сохранить word embeddings в файл (например, word_embeddings.txt):
with open("word_embeddings.txt", "w", encoding="utf-8") as f:
    for word, idx in vocab.items():
        vector = " ".join([f"{val:.6f}" for val in W1[idx]])
        f.write(f"{word} {vector}\n")
print("Word embeddings сохранены в 'word_embeddings.txt'.")

In [ ]:
# 8. Вычисление эмбеддингов для отзывов (документ-эмбеддинги = среднее эмбеддингов слов в отзыве)
# Результаты сохраняем в файл review_embeddings.csv с колонками: filename, label, embedding
with open("review_embeddings.csv", "w", encoding="utf-8", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["filename", "label", "embedding"])  # header
    for item in data:
        tokens = item["tokens"]
        # Для каждого слова достаём его эмбеддинг (если слово есть в словаре)
        vecs = [W1[vocab[token]] for token in tokens if token in vocab]
        if len(vecs) > 0:
            doc_embedding = np.mean(vecs, axis=0)
        else:
            doc_embedding = np.zeros(embedding_dim)
        # Представляем вектор в виде строки (числа разделены пробелом)
        embedding_str = " ".join([f"{val:.6f}" for val in doc_embedding.tolist()])
        writer.writerow([item["filename"], item["label"], embedding_str])
print("Эмбеддинги отзывов сохранены в 'review_embeddings.csv'.")